# 1. CONVERTING TXT TO DATAFRAME

In [19]:
import pandas as pd

In [20]:
tickers = ['AAPL', 'MSFT', 'AMZN', 'TSLA', 'GOOGL']

In [21]:
dataframes = []

for ticker in tickers:
    df = pd.read_csv("{}.us.txt".format(ticker))
    df['Date'] = pd.to_datetime(df['Date'])
    df['Ticker'] = ticker
    dataframes.append(df)  

full_df = pd.concat(dataframes)
full_df.set_index(['Ticker', 'Date'], inplace=True)   


In [22]:
full_df

Open        High         Low       Close    Volume  \
Ticker Date                                                                   
AAPL   1984-09-07     0.42388     0.42902     0.41874     0.42388  23220030   
       1984-09-10     0.42388     0.42516     0.41366     0.42134  18022532   
       1984-09-11     0.42516     0.43668     0.42516     0.42902  42498199   
       1984-09-12     0.42902     0.43157     0.41618     0.41618  37125801   
       1984-09-13     0.43927     0.44052     0.43927     0.43927  57822062   
...                       ...         ...         ...         ...       ...   
GOOGL  2017-11-06  1049.10000  1052.59000  1042.00000  1042.68000    913954   
       2017-11-07  1049.65000  1053.41000  1043.00000  1052.39000   1303832   
       2017-11-08  1050.05000  1062.69000  1047.05000  1058.29000   1214469   
       2017-11-09  1048.00000  1050.88000  1035.85000  1047.72000   1793994   
       2017-11-10  1043.87000  1046.63000  1041.22000  1044.15000    970498   

                   OpenInt  
Ticker Date                 
AAPL   1984-09-07        0  
       1984-09-10        0  
       1984-09-11        0  
       1984-09-12        0  
       1984-09-13        0  
...                    ...  
GOOGL  2017-11-06        0  
       2017-11-07        0  
       2017-11-08        0  
       2017-11-09        0  
       2017-11-10        0  

[26691 rows x 6 columns]

# 2. DATA TRANSFORM

## MISSING VALUE ADDITION

In [23]:
from datetime import datetime, timedelta

cutoff_date = pd.Timestamp('2007-11-10')  ### IMPORTANT CONCEPT
full_df = full_df.loc[full_df.index.get_level_values('Date') >= cutoff_date]  ### IMPORTANT CONCEPT

def fill_missing_values(ticker_df, ticker):
    full_dates = pd.date_range(start=ticker_df.index.min(), end=ticker_df.index.max(), freq='D')
    
    if isinstance(ticker_df.index, pd.MultiIndex) and 'Ticker' in ticker_df.index.names:
        ticker_df = ticker_df.droplevel('Ticker')
    
    ticker_df = ticker_df.reindex(full_dates) ### IMPORTANT CONCEPT
    
    ticker_df_interpolated = ticker_df.interpolate(method='linear')  ### IMPORTANT CONCEPT
    
    ticker_df_filled = ticker_df_interpolated.ffill()  ### IMPORTANT CONCEPT
    
    ticker_df_filled.index.name = 'Date'
    
    ticker_df_filled['Ticker'] = ticker
    
    ticker_df_filled.set_index('Ticker', append=True, inplace=True)
    
    ticker_df_filled = ticker_df_filled.reorder_levels(['Ticker', 'Date'])
    
    return ticker_df_filled

filled_dfs = []
for ticker in tickers:
    ticker_data = full_df.loc[ticker]                    
    filled_ticker_df = fill_missing_values(ticker_data, ticker)  
    filled_dfs.append(filled_ticker_df)                   

final_df = pd.concat(filled_dfs)                         
final_df.sort_index(inplace=True)                          



In [24]:
final_df

Open     High      Low    Close       Volume  OpenInt
Ticker Date                                                                
AAPL   2007-11-12   21.130   21.479   19.291   19.691  492362604.0      0.0
       2007-11-13   20.615   21.897   19.691   21.765  484373501.0      0.0
       2007-11-14   22.733   22.739   20.970   21.274  403585172.0      0.0
       2007-11-15   21.280   21.717   20.528   21.040  414487458.0      0.0
       2007-11-16   21.193   21.388   20.405   21.309  385660112.0      0.0
...                    ...      ...      ...      ...          ...      ...
TSLA   2017-11-06  307.000  307.500  299.010  302.780    6482486.0      0.0
       2017-11-07  301.020  306.500  300.030  306.050    5286320.0      0.0
       2017-11-08  305.500  306.890  301.300  304.310    4725510.0      0.0
       2017-11-09  302.500  304.460  296.300  302.990    5440335.0      0.0
       2017-11-10  302.500  308.360  301.850  302.990    4621912.0      0.0

[17301 rows x 6 columns]

# 3. DATA INTERPRETATION 

In [25]:
final_df['Daily Return'] = final_df.groupby(level='Ticker')['Close'].pct_change() * 100

final_df['MA_7'] = final_df.groupby(level='Ticker')['Close'].rolling(window=7).mean().reset_index(level=0, drop=True)

final_df['MA_30'] = final_df.groupby(level='Ticker')['Close'].rolling(window=30).mean().reset_index(level=0, drop=True)

final_df['Volatility_30'] = final_df.groupby(level='Ticker')['Daily Return'].rolling(window=30).std().reset_index(level=0, drop=True)

In [26]:
final_df

Open     High      Low    Close       Volume  OpenInt  \
Ticker Date                                                                   
AAPL   2007-11-12   21.130   21.479   19.291   19.691  492362604.0      0.0   
       2007-11-13   20.615   21.897   19.691   21.765  484373501.0      0.0   
       2007-11-14   22.733   22.739   20.970   21.274  403585172.0      0.0   
       2007-11-15   21.280   21.717   20.528   21.040  414487458.0      0.0   
       2007-11-16   21.193   21.388   20.405   21.309  385660112.0      0.0   
...                    ...      ...      ...      ...          ...      ...   
TSLA   2017-11-06  307.000  307.500  299.010  302.780    6482486.0      0.0   
       2017-11-07  301.020  306.500  300.030  306.050    5286320.0      0.0   
       2017-11-08  305.500  306.890  301.300  304.310    4725510.0      0.0   
       2017-11-09  302.500  304.460  296.300  302.990    5440335.0      0.0   
       2017-11-10  302.500  308.360  301.850  302.990    4621912.0      0.0   

                   Daily Return        MA_7       MA_30  Volatility_30  
Ticker Date                                                             
AAPL   2007-11-12           NaN         NaN         NaN            NaN  
       2007-11-13     10.532731         NaN         NaN            NaN  
       2007-11-14     -2.255915         NaN         NaN            NaN  
       2007-11-15     -1.099934         NaN         NaN            NaN  
       2007-11-16      1.278517         NaN         NaN            NaN  
...                         ...         ...         ...            ...  
TSLA   2017-11-06     -0.363078  309.944286  334.701044       1.977003  
       2017-11-07      1.079992  306.304286  333.317000       1.990582  
       2017-11-08     -0.568535  303.908571  332.029333       1.983164  
       2017-11-09     -0.433768  304.441429  330.276000       1.828429  
       2017-11-10      0.000000  303.998571  328.555667       1.830381  

[17301 rows x 10 columns]

In [27]:
full_df.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 11934 entries, ('AAPL', Timestamp('2007-11-12 00:00:00')) to ('GOOGL', Timestamp('2017-11-10 00:00:00'))
Data columns (total 6 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   Open     11934 non-null  float64
 1   High     11934 non-null  float64
 2   Low      11934 non-null  float64
 3   Close    11934 non-null  float64
 4   Volume   11934 non-null  int64  
 5   OpenInt  11934 non-null  int64  
dtypes: float64(4), int64(2)
memory usage: 918.1+ KB


In [28]:
final_df.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 17301 entries, ('AAPL', Timestamp('2007-11-12 00:00:00')) to ('TSLA', Timestamp('2017-11-10 00:00:00'))
Data columns (total 10 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Open           17301 non-null  float64
 1   High           17301 non-null  float64
 2   Low            17301 non-null  float64
 3   Close          17301 non-null  float64
 4   Volume         17301 non-null  float64
 5   OpenInt        17301 non-null  float64
 6   Daily Return   17296 non-null  float64
 7   MA_7           17271 non-null  float64
 8   MA_30          17156 non-null  float64
 9   Volatility_30  17151 non-null  float64
dtypes: float64(10)
memory usage: 1.5+ MB


# 4. EDA

In [29]:
final_df

Open     High      Low    Close       Volume  OpenInt  \
Ticker Date                                                                   
AAPL   2007-11-12   21.130   21.479   19.291   19.691  492362604.0      0.0   
       2007-11-13   20.615   21.897   19.691   21.765  484373501.0      0.0   
       2007-11-14   22.733   22.739   20.970   21.274  403585172.0      0.0   
       2007-11-15   21.280   21.717   20.528   21.040  414487458.0      0.0   
       2007-11-16   21.193   21.388   20.405   21.309  385660112.0      0.0   
...                    ...      ...      ...      ...          ...      ...   
TSLA   2017-11-06  307.000  307.500  299.010  302.780    6482486.0      0.0   
       2017-11-07  301.020  306.500  300.030  306.050    5286320.0      0.0   
       2017-11-08  305.500  306.890  301.300  304.310    4725510.0      0.0   
       2017-11-09  302.500  304.460  296.300  302.990    5440335.0      0.0   
       2017-11-10  302.500  308.360  301.850  302.990    4621912.0      0.0   

                   Daily Return        MA_7       MA_30  Volatility_30  
Ticker Date                                                             
AAPL   2007-11-12           NaN         NaN         NaN            NaN  
       2007-11-13     10.532731         NaN         NaN            NaN  
       2007-11-14     -2.255915         NaN         NaN            NaN  
       2007-11-15     -1.099934         NaN         NaN            NaN  
       2007-11-16      1.278517         NaN         NaN            NaN  
...                         ...         ...         ...            ...  
TSLA   2017-11-06     -0.363078  309.944286  334.701044       1.977003  
       2017-11-07      1.079992  306.304286  333.317000       1.990582  
       2017-11-08     -0.568535  303.908571  332.029333       1.983164  
       2017-11-09     -0.433768  304.441429  330.276000       1.828429  
       2017-11-10      0.000000  303.998571  328.555667       1.830381  

[17301 rows x 10 columns]

In [30]:
avg_return = final_df.groupby('Ticker')['Daily Return'].mean()
avg_return

Ticker
AAPL     0.070917
AMZN     0.091275
GOOGL    0.042695
MSFT     0.039124
TSLA     0.138318
Name: Daily Return, dtype: float64

In [33]:
highest_avg_return_stock = avg_return.idxmax()
highest_avg_return_value = avg_return.max()

In [37]:
print(f"Stock with highest average daily return: {highest_avg_return_stock} ({highest_avg_return_value:})")

Stock with highest average daily return: TSLA (0.13831838002518207)


In [ ]:
df['YearMonth'] = df['Date'].dt.to_period('M')